In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
device = torch.device("cuda")   
    
    
batch_size = 128
model_name = "VGG16_quant_project"
model = VGG16_quant_project()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
print_freq = 100
def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch,adjust_list):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1

In [3]:
lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=weight_decay)

if not os.path.exists('result'): 
    os.makedirs('result')

fdir = 'result/'+str(model_name) 
if not os.path.exists(fdir):
    os.makedirs(fdir)

adjust_list = [80,90]

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch,adjust_list)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/391]	Time 0.412 (0.412)	Data 0.251 (0.251)	Loss 2.4524 (2.4524)	Prec 10.938% (10.938%)
Epoch: [0][100/391]	Time 0.054 (0.058)	Data 0.002 (0.004)	Loss 2.2622 (3.4320)	Prec 8.594% (10.582%)
Epoch: [0][200/391]	Time 0.055 (0.057)	Data 0.002 (0.003)	Loss 2.0569 (2.8186)	Prec 17.188% (13.254%)
Epoch: [0][300/391]	Time 0.060 (0.056)	Data 0.002 (0.003)	Loss 2.0206 (2.5567)	Prec 21.875% (15.695%)
Validation starts
Test: [0/79]	Time 0.205 (0.205)	Loss 1.9066 (1.9066)	Prec 29.688% (29.688%)
 * Prec 24.470% 
best acc: 24.470000
Epoch: [1][0/391]	Time 0.308 (0.308)	Data 0.257 (0.257)	Loss 1.9837 (1.9837)	Prec 21.094% (21.094%)
Epoch: [1][100/391]	Time 0.058 (0.057)	Data 0.002 (0.004)	Loss 1.9341 (1.9230)	Prec 18.750% (22.741%)
Epoch: [1][200/391]	Time 0.054 (0.056)	Data 0.003 (0.003)	Loss 1.8041 (1.9032)	Prec 31.250% (23.865%)
Epoch: [1][300/391]	Time 0.053 (0.056)	Data 0.001 (0.002)	Loss 1.9593 (1.8844)	Prec 17.969% (24.886%)
Validation starts
Test: [0/79]	Time 0.235 (0.235)	Loss 1.9

Epoch: [15][200/391]	Time 0.053 (0.057)	Data 0.002 (0.003)	Loss 0.6900 (0.5536)	Prec 76.562% (81.207%)
Epoch: [15][300/391]	Time 0.054 (0.057)	Data 0.001 (0.002)	Loss 0.5596 (0.5621)	Prec 78.125% (80.915%)
Validation starts
Test: [0/79]	Time 0.229 (0.229)	Loss 0.5778 (0.5778)	Prec 80.469% (80.469%)
 * Prec 78.930% 
best acc: 78.930000
Epoch: [16][0/391]	Time 0.308 (0.308)	Data 0.260 (0.260)	Loss 0.6491 (0.6491)	Prec 78.125% (78.125%)
Epoch: [16][100/391]	Time 0.053 (0.058)	Data 0.002 (0.004)	Loss 0.4911 (0.5290)	Prec 81.250% (81.915%)
Epoch: [16][200/391]	Time 0.056 (0.057)	Data 0.002 (0.003)	Loss 0.5030 (0.5356)	Prec 82.812% (81.573%)
Epoch: [16][300/391]	Time 0.064 (0.056)	Data 0.002 (0.003)	Loss 0.5472 (0.5347)	Prec 80.469% (81.689%)
Validation starts
Test: [0/79]	Time 0.220 (0.220)	Loss 0.6140 (0.6140)	Prec 79.688% (79.688%)
 * Prec 78.650% 
best acc: 78.930000
Epoch: [17][0/391]	Time 0.288 (0.288)	Data 0.239 (0.239)	Loss 0.4693 (0.4693)	Prec 84.375% (84.375%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.058 (0.056)	Data 0.002 (0.002)	Loss 0.3177 (0.3149)	Prec 85.938% (89.348%)
Validation starts
Test: [0/79]	Time 0.194 (0.194)	Loss 0.3138 (0.3138)	Prec 89.844% (89.844%)
 * Prec 84.310% 
best acc: 84.470000
Epoch: [31][0/391]	Time 0.312 (0.312)	Data 0.263 (0.263)	Loss 0.1730 (0.1730)	Prec 93.750% (93.750%)
Epoch: [31][100/391]	Time 0.056 (0.058)	Data 0.001 (0.004)	Loss 0.2912 (0.2980)	Prec 90.625% (90.231%)
Epoch: [31][200/391]	Time 0.066 (0.057)	Data 0.002 (0.003)	Loss 0.3191 (0.3014)	Prec 91.406% (89.887%)
Epoch: [31][300/391]	Time 0.053 (0.057)	Data 0.002 (0.003)	Loss 0.4346 (0.3064)	Prec 86.719% (89.693%)
Validation starts
Test: [0/79]	Time 0.247 (0.247)	Loss 0.3946 (0.3946)	Prec 87.500% (87.500%)
 * Prec 84.860% 
best acc: 84.860000
Epoch: [32][0/391]	Time 0.305 (0.305)	Data 0.261 (0.261)	Loss 0.3022 (0.3022)	Prec 90.625% (90.625%)
Epoch: [32][100/391]	Time 0.056 (0.058)	Data 0.002 (0.004)	Loss 0.3824 (0.2789)	Prec 89.062% (90.416%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.223 (0.223)	Loss 0.3133 (0.3133)	Prec 90.625% (90.625%)
 * Prec 86.920% 
best acc: 87.210000
Epoch: [46][0/391]	Time 0.265 (0.265)	Data 0.218 (0.218)	Loss 0.1727 (0.1727)	Prec 94.531% (94.531%)
Epoch: [46][100/391]	Time 0.051 (0.057)	Data 0.002 (0.004)	Loss 0.1823 (0.1945)	Prec 92.969% (93.394%)
Epoch: [46][200/391]	Time 0.061 (0.057)	Data 0.001 (0.003)	Loss 0.1787 (0.1937)	Prec 92.188% (93.342%)
Epoch: [46][300/391]	Time 0.064 (0.056)	Data 0.001 (0.002)	Loss 0.2201 (0.2019)	Prec 89.844% (93.062%)
Validation starts
Test: [0/79]	Time 0.245 (0.245)	Loss 0.3573 (0.3573)	Prec 87.500% (87.500%)
 * Prec 85.060% 
best acc: 87.210000
Epoch: [47][0/391]	Time 0.289 (0.289)	Data 0.242 (0.242)	Loss 0.2177 (0.2177)	Prec 91.406% (91.406%)
Epoch: [47][100/391]	Time 0.056 (0.058)	Data 0.001 (0.004)	Loss 0.1460 (0.1930)	Prec 94.531% (93.557%)
Epoch: [47][200/391]	Time 0.056 (0.057)	Data 0.001 (0.003)	Loss 0.1566 (0.1956)	Prec 92.969% (93.369%)
Epoch: [47][300/391]	

 * Prec 86.080% 
best acc: 88.140000
Epoch: [61][0/391]	Time 0.299 (0.299)	Data 0.247 (0.247)	Loss 0.0937 (0.0937)	Prec 97.656% (97.656%)
Epoch: [61][100/391]	Time 0.059 (0.058)	Data 0.001 (0.004)	Loss 0.1269 (0.1482)	Prec 95.312% (94.964%)
Epoch: [61][200/391]	Time 0.056 (0.057)	Data 0.002 (0.003)	Loss 0.1110 (0.1423)	Prec 96.875% (95.145%)
Epoch: [61][300/391]	Time 0.056 (0.056)	Data 0.002 (0.002)	Loss 0.0617 (0.1459)	Prec 97.656% (95.043%)
Validation starts
Test: [0/79]	Time 0.229 (0.229)	Loss 0.2397 (0.2397)	Prec 90.625% (90.625%)
 * Prec 87.100% 
best acc: 88.140000
Epoch: [62][0/391]	Time 0.302 (0.302)	Data 0.253 (0.253)	Loss 0.1530 (0.1530)	Prec 95.312% (95.312%)
Epoch: [62][100/391]	Time 0.058 (0.058)	Data 0.002 (0.004)	Loss 0.1306 (0.1343)	Prec 96.875% (95.359%)
Epoch: [62][200/391]	Time 0.054 (0.058)	Data 0.002 (0.003)	Loss 0.0606 (0.1377)	Prec 97.656% (95.285%)
Epoch: [62][300/391]	Time 0.058 (0.057)	Data 0.002 (0.003)	Loss 0.1315 (0.1432)	Prec 96.094% (95.063%)
Validation s

Epoch: [76][100/391]	Time 0.055 (0.058)	Data 0.002 (0.004)	Loss 0.0898 (0.1089)	Prec 96.875% (96.310%)
Epoch: [76][200/391]	Time 0.055 (0.057)	Data 0.001 (0.003)	Loss 0.0978 (0.1118)	Prec 98.438% (96.226%)
Epoch: [76][300/391]	Time 0.056 (0.056)	Data 0.001 (0.002)	Loss 0.0760 (0.1169)	Prec 96.875% (96.140%)
Validation starts
Test: [0/79]	Time 0.233 (0.233)	Loss 0.3449 (0.3449)	Prec 92.188% (92.188%)
 * Prec 86.700% 
best acc: 88.740000
Epoch: [77][0/391]	Time 0.279 (0.279)	Data 0.232 (0.232)	Loss 0.1696 (0.1696)	Prec 94.531% (94.531%)
Epoch: [77][100/391]	Time 0.053 (0.057)	Data 0.001 (0.004)	Loss 0.1624 (0.1200)	Prec 93.750% (96.040%)
Epoch: [77][200/391]	Time 0.056 (0.056)	Data 0.001 (0.003)	Loss 0.1407 (0.1219)	Prec 95.312% (95.927%)
Epoch: [77][300/391]	Time 0.053 (0.056)	Data 0.001 (0.002)	Loss 0.1175 (0.1209)	Prec 95.312% (95.948%)
Validation starts
Test: [0/79]	Time 0.252 (0.252)	Loss 0.1998 (0.1998)	Prec 92.188% (92.188%)
 * Prec 86.050% 
best acc: 88.740000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.055 (0.056)	Data 0.001 (0.003)	Loss 0.0072 (0.0177)	Prec 100.000% (99.390%)
Epoch: [91][300/391]	Time 0.056 (0.056)	Data 0.001 (0.002)	Loss 0.0057 (0.0176)	Prec 100.000% (99.421%)
Validation starts
Test: [0/79]	Time 0.253 (0.253)	Loss 0.1996 (0.1996)	Prec 96.094% (96.094%)
 * Prec 91.380% 
best acc: 91.380000
Epoch: [92][0/391]	Time 0.272 (0.272)	Data 0.230 (0.230)	Loss 0.0136 (0.0136)	Prec 99.219% (99.219%)
Epoch: [92][100/391]	Time 0.054 (0.057)	Data 0.001 (0.004)	Loss 0.0074 (0.0162)	Prec 100.000% (99.497%)
Epoch: [92][200/391]	Time 0.053 (0.056)	Data 0.001 (0.003)	Loss 0.0251 (0.0158)	Prec 99.219% (99.499%)
Epoch: [92][300/391]	Time 0.061 (0.056)	Data 0.001 (0.002)	Loss 0.0315 (0.0166)	Prec 99.219% (99.496%)
Validation starts
Test: [0/79]	Time 0.233 (0.233)	Loss 0.1921 (0.1921)	Prec 96.094% (96.094%)
 * Prec 91.380% 
best acc: 91.380000
Epoch: [93][0/391]	Time 0.284 (0.284)	Data 0.237 (0.237)	Loss 0.0277 (0.0277)	Prec 98.438% (98.438%)
Epoch: [93][100/39

In [3]:
PATH = "result/VGG16_quant_project/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)



Test set: Accuracy: 9157/10000 (92%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
41 -th layer prehooked
46 -th layer prehooked
50 -th layer prehooked
54 -th layer prehooked


In [177]:
w_bit = 4
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta

x_bit = 4
x = save_output.outputs[8][0]
x_alpha = model.features[27].act_alpha
x_delta = x_alpha/(2**x_bit-1)
act_quant = act_quantization(x_bit)
x_q = act_quant(x,x_alpha)
x_int = x_q/x_delta

conv_int = nn.Conv2d(8,8,kernel_size=3, padding=1, bias=False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
output_int = conv_int(x_int)
psum_recovered = output_int*w_delta*x_delta
relu = nn.ReLU(inplace=True)
psum_after_relu = relu(psum_recovered)

difference = (save_output.outputs[9][0] - psum_after_relu).mean()

print(difference)

tensor(-1.8603e-09, device='cuda:0', grad_fn=<MeanBackward0>)


In [178]:
padding = 1
stride = 1
array_size = 8
x_size = x_int.size()
nig = range(x_size[2])
njg = range(x_size[3])

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

a_pad = torch.zeros((x_size[1],x_size[2]+2*padding,x_size[3]+2*padding))
act_int = x_int[0,:,:,:]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = act_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))

ic_tile = range(int(int(act_int.size(0))/array_size))
oc_tile = range(int(int(w_int.size(0))/array_size))


a_tile = torch.zeros(len(ic_tile),array_size,len(nig)+padding*2,len(njg)+padding*2).cuda()
a_tile = torch.reshape(a_tile,(a_tile.size(0),a_tile.size(1),-1))

for ict in ic_tile:
    a_tile[ict,:,:] = a_pad[(ict*array_size):((ict+1)*array_size),:]

w_tile = torch.zeros(len(ic_tile),len(oc_tile),array_size,array_size,len(kijg)).cuda()



for oct in oc_tile:
    for ict in ic_tile:
        w_tile[ict,oct,:,:,:] = w_int[(oct*array_size):((oct+1)*array_size),(ict*array_size):((ict+1)*array_size),:]

        
        
p_nijg = range(a_pad.size(1)) ## paded activation's nij group

psum = torch.zeros(len(ic_tile),len(oc_tile),array_size,len(p_nijg),len(kijg)).cuda() 

for kij in kijg:
    for ict in ic_tile:
        for oct in oc_tile:
            for nij in p_nijg:     # time domain, sequentially given input
                m = nn.Linear(array_size, array_size, bias=False)
                m.weight = torch.nn.Parameter(w_tile[ict,oct,:,:,kij])
                psum[ict,oct,:, nij, kij] = m(a_tile[ict,:,nij]).cuda()
                


In [179]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32 + 2*pad = 34

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1) #34 - 2 - 1 + 1 = 32
o_nijg = range(o_ni_dim**2)    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:  
        for ict in ic_tile:
            for oct in oc_tile:
                out[oct*array_size:(oct+1)*array_size,o_nij] = out[oct*array_size:(oct+1)*array_size,o_nij] + \
                psum[ict,oct,:, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 2nd index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

In [180]:
## Helper functions to dump files
def dec_to_bin(arr,bit):
    bin_arr = []
    for a in arr:
        a = int(a)
        if a < 0:
            a+=bit
        b = '{0:04b}'.format(int(a))
        bin_arr.append(b)
    return bin_arr

def dec_to_bin_psum(arr,bit):
    bin_arr = []
    for a in arr:
        a = int(a)
        sign = 0
        if a < 0:
            sign = 1
            a+=bit
        b = '{0:16b}'.format(int(a))
        if sign == 1:
            b = b.replace(" ", "1")
        else:
            b = b.replace(" ","0")
        bin_arr.append(b)
    return bin_arr


def convert_to_list(arr):
    return arr.tolist()



In [181]:
## Activation dump

fp_act = open('activation_project.txt','w')
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act_dec = open('activation_dec.txt','w')
for i in range(36):
    act_line = a_pad[:,i]
    act_arr = [int(j+0.001) for j in convert_to_list(act_line)]
    for a in act_arr[::-1]:
        #print(int(a))
        fp_act_dec.write(str(int(a)))
        fp_act_dec.write(" ")
    fp_act_dec.write("\n")
    bin_act = dec_to_bin(act_arr,16)
    for b in bin_act[::-1]:
        #print(b)
        fp_act.write(b)
    fp_act.write('\n')
    

fp_act.close()
fp_act_dec.close()

In [182]:
## Weight dump
fp_wgt = open('weight_project.txt','w')
fp_wgt.write("#####\n")
fp_wgt.write("#####\n")
fp_wgt.write("#####\n")
fp_wgt_dec = open('weight_dec.txt','w')

for kij in range(9):
    for w in range(8):
        w_line = w_int[w,:,kij]
        w_arr = []
        for i in convert_to_list(w_line):
            if i < 0:
                w_arr.append(int(i-0.001))
            else:
                w_arr.append(int(i+0.001))
        
        for ww in w_arr[::-1]:
            #print(int(ww))
            fp_wgt_dec.write(str(int(ww)))
            fp_wgt_dec.write(" ")
        fp_wgt_dec.write("\n")
        bin_wgt = dec_to_bin(w_arr,16)
        for b in bin_wgt[::-1]:
            #print("Count = {} , b = {}".format(count,b))
            fp_wgt.write(b)
        fp_wgt.write('\n')

fp_wgt.close()
fp_wgt_dec.close()

In [183]:
## psum dump
fp_psum = open('psum_project.txt','w')
fp_psum.write("#####\n")
fp_psum.write("#####\n")
fp_psum.write("#####\n")
fp_psum_dec = open('psum_dec.txt','w')
fp_psum_relu = open('output_project.txt','w')
for kij in range(16):
    psum_line = out[:,kij]
    psum_arr = []
    out_arr = []
    for i in convert_to_list(psum_line):
        if i < 0:
            psum_arr.append(int(i-0.001))
            out_arr.append(0)
        else:
            psum_arr.append(int(i+0.001))
            out_arr.append(int(i+0.001))
    for p in psum_arr:
        #print(int(a))
        fp_psum_dec.write(str(int(p)))
        fp_psum_dec.write(" ")
    fp_psum_dec.write("\n")
    bin_act = dec_to_bin_psum(psum_arr,65536)
    out_bin_act = dec_to_bin_psum(out_arr,65536)
    for b in bin_act:
        #print(b)
        fp_psum.write(b)
        fp_psum.write(' ')
    fp_psum.write('\n')
    fp_psum_relu.write("#")
    fp_psum_relu.write("#")
    
    for b in out_bin_act:
        fp_psum_relu.write(b)
        fp_psum_relu.write(' ')
    fp_psum_relu.write("\n")
fp_psum.close()
fp_psum_dec.close()
fp_psum_relu.close()